In [1]:
from scipy.io import wavfile
from scipy.signal import butter, filtfilt
import numpy as np
import librosa
import matplotlib.pyplot as plt

In [2]:
def read_audio_file(file_path):
    """Đọc file âm thanh và trả về tỷ lệ mẫu và dữ liệu âm thanh đã chuẩn hóa."""
    sample_rate, audio_data = wavfile.read(file_path)
    audio_data = audio_data / np.max(np.abs(audio_data))  # Chuẩn hóa dữ liệu âm thanh về phạm vi từ -1 đến 1
    return sample_rate, audio_data

In [3]:
def apply_lowpass_filter(audio_data, sample_rate, cutoff_freq, order):
    """Áp dụng bộ lọc thông thấp cho dữ liệu âm thanh."""
    b, a = butter(order, cutoff_freq / (sample_rate / 2), btype='low', analog=False)
    filtered_audio_data = filtfilt(b, a, audio_data)
    return filtered_audio_data

In [4]:
def apply_highpass_filter(audio_data, sample_rate, cutoff_freq, order):
    """Áp dụng bộ lọc thông cao cho dữ liệu âm thanh."""
    b, a = butter(order, cutoff_freq / (sample_rate / 2), btype='high', analog=False)
    filtered_audio_data = filtfilt(b, a, audio_data)
    return filtered_audio_data

In [5]:
def save_audio_file(output_file, sample_rate, audio_data):
    """Lưu dữ liệu âm thanh vào file mới."""
    wavfile.write(output_file, sample_rate, np.array(audio_data * (2**15 - 1), dtype=np.int16))

In [6]:
# def normalize_sample_rate(audio_data, orig_sr, target_sr):
#     if orig_sr != target_sr:
#         audio_data = librosa.resample(audio_data, target_sr)
#     return audio_data

In [7]:
from scipy.signal import resample

def normalize_sample_rate(audio_data, orig_sr, target_sr):
    if orig_sr != target_sr:
        # Tính toán số lượng mẫu mới dựa trên tỷ lệ mẫu mới
        num_samples = int(round(len(audio_data) * float(target_sr) / orig_sr))
        # Resample âm thanh
        resampled_audio = resample(audio_data, num_samples)
        return resampled_audio, target_sr
    else:
        return audio_data, target_sr

In [8]:
def normalize_audio(audio, target_dBFS):
    current_dBFS = librosa.amplitude_to_db(np.max(np.abs(audio)), ref=1.0)

    # Tính toán độ chênh lệch giữa mức âm thanh hiện tại và mức âm thanh mục tiêu
    dBFS_change = target_dBFS - current_dBFS

    # Chuẩn hóa âm thanh bằng cách thay đổi biên độ
    normalized_audio = audio * (10.0 ** (dBFS_change / 20.0))
    return normalized_audio

In [9]:
def plot_audio(audio_data, sample_rate, filtered_audio_data):
    """Hiển thị âm thanh gốc và âm thanh đã lọc."""
    plt.figure(figsize=(12, 6))
    plt.plot(np.arange(len(audio_data)) / sample_rate, audio_data, label='Original Audio')
    plt.plot(np.arange(len(filtered_audio_data)) / sample_rate, filtered_audio_data, label='Filtered Audio (Thresholded)')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title('Original vs. Filtered Audio')
    plt.legend()
    plt.grid(True)
    plt.show()

In [10]:
def normalize_audio_file(output_folder_path, file_name, file_path, cutoff_freq_low, cutoff_freq_high, order, target_sr, target_dBFS):
    sample_rate, audio_data = read_audio_file(file_path)

    # audio_data, sample_rate = normalize_sample_rate(audio_data, sample_rate, target_sr)

    audio_data = normalize_audio(audio_data, target_dBFS)

    filtered_audio_data = apply_lowpass_filter(audio_data, sample_rate, cutoff_freq_low, order)
    filtered_audio_data = apply_highpass_filter(audio_data, sample_rate, cutoff_freq_high, order)

    # plot_audio(audio_data, sample_rate, filtered_audio_data)
    # # Lưu dữ liệu âm thanh đã lọc vào file mới
    output_file = f'{output_folder_path}/cleaned_{file_name}.wav'
    save_audio_file(output_file, sample_rate, filtered_audio_data)

    # # Hiển thị âm thanh gốc và âm thanh đã lọc
    # plot_audio(audio_data, sample_rate, filtered_audio_data)

In [11]:
normalize_audio_file("", "normalized_audio", r"D:\data_analysis\speech_emotion_recognition\audio_test\Bong_Dung_Muon_Khoc\divide_segment\1\Sad\segment_745.wav", 800, 100, 5, 16000, -20)

In [12]:
import os

In [13]:
def create_output_folder(output_folder):
        """
        Kiểm tra và tạo thư mục đầu ra nếu nó không tồn tại.
        """
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

In [14]:
def process_folder(input_folder_path, output_folder_path):
    """
    Xử lý tất cả các tệp WAV trong thư mục để tìm các phân đoạn nói.
    """
    for file_name in os.listdir(input_folder_path):
        try:
            if file_name.endswith('.wav'):
                file_path = os.path.join(input_folder_path, file_name)
                create_output_folder(output_folder_path)
                normalize_audio_file(output_folder_path, file_name.replace('.wav', ''), file_path, 2000, 100, 5, 48000, -20)
        except Exception as e:
            print(f"An error occurred while processing file '{file_name}': {e}")

In [15]:
def process_folders(input_folder, output_folder):
    for root, dirs, files in os.walk(input_folder):
        for folder in dirs:
            subdirectory_input = os.path.join(root, folder)

            relative_path = os.path.relpath(subdirectory_input, input_folder)
            subdirectory_output = os.path.join(output_folder, relative_path)

            if not os.path.exists(subdirectory_output):
                os.makedirs(subdirectory_output)
            process_folder(subdirectory_input,subdirectory_output)

In [16]:
process_folders(r'D:\data_analysis\speech_emotion_recognition\data\EnglishDataset\cleaned_data', r'D:\data_analysis\speech_emotion_recognition\data\EnglishDataset\normalized_data')

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10528\2549505823.py:4: RuntimeWarning: invalid value encountered in divide
  audio_data = audio_data / np.max(np.abs(audio_data))  # Chuẩn hóa dữ liệu âm thanh về phạm vi từ -1 đến 1
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10528\1610263151.py:3: RuntimeWarning: invalid value encountered in cast
  wavfile.write(output_file, sample_rate, np.array(audio_data * (2**15 - 1), dtype=np.int16))
